In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

# AWS S3 bucket name
AWS_S3_BUCKET = "user-0a8597384a69-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/0a8597384a69-bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)


--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-3177652997416107> in <module> 
 29 SOURCE_URL = "s3n://{0}:{1}@{2}" . format ( ACCESS_KEY , ENCODED_SECRET_KEY , AWS_S3_BUCKET ) 
 30 # Mount the drive 
 ---> 31 dbutils . fs . mount ( SOURCE_URL , MOUNT_NAME ) 
 32 
 33 display ( dbutils . fs . ls ( "/mnt/mount_name/" ) ) 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 387 exc . __context__ = None 
 388 exc . __cause__ = None 
 --> 389 raise exc
 390 
 391 return f_with_exception_handling

 ExecutionError : An error occurred while calling o404.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/0a8597384a69-bucket; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/0a8597384a69-bucket
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:756)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:776)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:750)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/0a8597384a69-bucket
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:536)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$2(MetadataManager.scala:904)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:685)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:893)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:544)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:120)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:54)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:53)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:53)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$4.$anonfun$applyOrElse$5(DbfsServerBackend.scala:368)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:421)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:196)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:419)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:413)
	at com.databricks.rpc.ServerBackend.withAttributionContext(ServerBackend.scala:22)
	at com.datab

In [0]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location = "/mnt/0a8597384a69-bucket/topics/0a8597384a69.{}/partition=0/*.json" 
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_pin = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location.format("pin"))
df_geo = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location.format("geo"))
df_user = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location.format("user"))
# Display Spark dataframe to check its content
display(df_pin)
display(df_geo)
display(df_user)

age,date_joined,first_name,index,last_name
21,2015-11-10T09:27:42,Andrea,8731,Alexander
24,2016-03-31T20:56:39,Austin,8887,Rodriguez
36,2015-12-20T16:38:13,Michelle,4315,Prince
32,2017-10-10T20:09:33,Christian,10625,Lang
22,2016-02-11T20:46:04,Jennifer,9672,Hudson
32,2016-04-02T03:51:23,Brittany,1313,Jones
34,2016-12-22T00:02:02,Thomas,10794,Turner
20,2016-01-07T19:49:22,David,2959,Griffith
26,2015-12-20T10:28:00,Brendan,9875,Joseph
21,2016-01-03T15:42:12,Annette,2074,Forbes


index,latitude,longitude,timestamp
10794,-89.5236,-154.567,2022-01-01T02:26:50
10625,-84.4944,-81.0613,2018-07-13T11:51:15
7528,-89.9787,-173.293,2020-08-28T03:52:47
2863,-5.34445,-177.924,2020-04-27T13:34:16
5494,-82.6768,-129.202,2021-07-21T02:02:35
5069,-63.0063,-157.474,2021-03-20T09:32:44
2923,-84.6302,-164.507,2019-09-08T22:53:09
3089,-89.9787,-173.293,2018-02-28T05:31:29
6063,-89.1797,-174.015,2021-07-20T09:02:47
3454,-0.375174,49.8106,2021-07-25T02:20:29


age,date_joined,first_name,index,last_name
21,2015-11-10T09:27:42,Andrea,8731,Alexander
24,2016-03-31T20:56:39,Austin,8887,Rodriguez
36,2015-12-20T16:38:13,Michelle,4315,Prince
32,2017-10-10T20:09:33,Christian,10625,Lang
22,2016-02-11T20:46:04,Jennifer,9672,Hudson
32,2016-04-02T03:51:23,Brittany,1313,Jones
34,2016-12-22T00:02:02,Thomas,10794,Turner
20,2016-01-07T19:49:22,David,2959,Griffith
26,2015-12-20T10:28:00,Brendan,9875,Joseph
21,2016-01-03T15:42:12,Annette,2074,Forbes
